<a href="https://colab.research.google.com/github/have-not/github-aws-cicd/blob/master/AutoFriends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tweepy

In [ ]:
import time
import tweepy
import random
import os
import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
consumer_key = '' # Consumer API key
consumer_secret = '' # Consumer API Secret key
access_token_key = '' # Access token
access_token_secret = '' # Access token secret

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token_key, access_token_secret)
api = tweepy.API(auth)

In [ ]:
my_user_id = ''

In [ ]:
new_dir_path_recursive = '/content/data/Twitter/friend/'
if not os.path.exists(new_dir_path_recursive):
    os.makedirs(new_dir_path_recursive)

In [ ]:
def save_id(id, filepath):
    with open(filepath, 'a') as f:
        print(id, file=f)

In [ ]:
def get_ids_fromfile(filepath):
    datalist = []
    if os.path.exists(filepath):
        with open(filepath, 'r') as f:
            datalist = f.readlines()
    return datalist

In [ ]:
def get_followers_ids():
    followers_ids = []
    try:
          followers_ids = api.followers_ids()
    except tweepy.RateLimitError as e:
        raise e
    except tweepy.TweepError as e:
        raise e
    else:
        return followers_ids

In [ ]:
def norefollow_ids(filepath):
    try:
        #特定日のフォローがまだ片思いの人の一覧を取得
        followers_ids = get_followers_ids()
        temp_friends_ids = get_ids_fromfile(filepath)

        #特定日のフォローがまだ片思いの人の一覧を取得
        ids = list(set(temp_friends_ids) - set(followers_ids))
    except tweepy.RateLimitError as e:
        raise e
    except tweepy.TweepError as e:
        raise e
    else:
        return ids

In [ ]:
def remove_friends(filepath):
    for i in range(3):
        try:
            ids = norefollow_ids(filepath)
        except tweepy.RateLimitError as e:
            print(e)
            time.sleep(15 * 60)
        except tweepy.TweepError as e:
            print(e)
            time.sleep(5 * 60)
            break
        else:
            time.sleep(1)
            break


    for id in ids:
        for i in range(3):
            try:
                api.destroy_friendship(id)
                print(str(id) + 'をリムーブ')
                time.sleep(1)
            except tweepy.RateLimitError as e:
                print(e)
                time.sleep(15 * 60)
            except tweepy.TweepError as e:
                print(e)
                time.sleep(5 * 60)
                break
            else:
                time.sleep(1)
                break

    #ファイルを削除

In [ ]:
def delete_retweet(id):
    try:
      status = api.get_status(ID, include_my_retweet=1)
      if status.retweeted == True:
          api.destroy_status(status.current_user_retweet['id'])
    except tweepy.RateLimitError as e:
        raise e
    except tweepy.TweepError as e:
        raise e

In [ ]:
def delete_retweets(filepath):
    #過去のリツイートのツイートIDを取得
    old_tweet_ids = get_ids_fromfile(filepath)

    for tweet_id in old_tweet_ids:
        for i in range(3):
            try:
                  delete_retweet(id)
                  print('リツイート削除に成功しました')
            except tweepy.RateLimitError as e:
                print(e)
                time.sleep(15 * 60)
            except tweepy.TweepError as e:
                print(e)
                time.sleep(5 * 60)
                break
            else:
                time.sleep(1)
                break
                
    #ファイルを削除

In [ ]:
 def search_tweets(q, sinceDate):
    for i in range(3):
        try:
          results = tweepy.Cursor(api.search, q=q, tweet_mode='extended', since = sinceDate, result_type="mixed", include_entities=True).items()
        except tweepy.RateLimitError as e:
            print(e)
            time.sleep(15 * 60)
        except tweepy.TweepError as e:
            print(e)
            time.sleep(5 * 60)
            break
        else:
            return results

In [ ]:
def create_frend(q, todaystr, sinceDate):

    # Twitterの検索結果を取得
    results = search_tweets(q, sinceDate)

    for result in results:
        screen_name = result.user.screen_name
        is_following = result.user._json['following']
        if screen_name == my_user_id:
            print('自分')
        else:
            if is_following:
                print('@' + screen_name + 'はフォロー済み')
                time.sleep(30)
            else:
                print('@' + screen_name + 'をフォロー処理中...')
                for i in range(3): #RateLimitにかかったら3回までリトライ
                    try:
                        api.create_friendship(screen_name)
                        save_id(result.user.id, '/content/data/Twitter/friend/' + todaystr)
                        api.retweet(result.id)
                        save_id(result.user.id, '/content/data/Twitter/retweet/' + todaystr)
                        print('フォローに成功しました')
                    except tweepy.RateLimitError as e:
                        print(e)
                        time.sleep(15 * 60)
                    except tweepy.TweepError as e:
                        print(e)
                        time.sleep(5 * 60)
                        break
                    else:
                        time.sleep(3 * 60)
                        break

In [ ]:
def main():

    while True:
    #今日日付と過去の一覧を取得したい日付を設定
        today = datetime.date.today()
        todaystr = today.strftime('%Y%m%d')
        pastdaystr = (today - datetime.timedelta(days=3)).strftime('%Y%m%d')

        delete_retweets('/content/data/Twitter/retweet/' + pastdaystr)

        remove_friends('/content/data/Twitter/friend/' + pastdaystr)

        q = f"フォロバ100%20おはよ%20-filter%3Areplies%20-filter%3Aretweets"
        create_frend(q, todaystr, today - datetime.timedelta(days=1))

if __name__ == "__main__":
    main()